In [1]:
import os
from tqdm.notebook import tqdm
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Existing dataset dir
data_dir = './data/'

# Existing plots dir
plots_dir = data_dir+'/plots/'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)


In [3]:
file_settings = {
    'ethereum': {  'nr': '1', 'file_name': './data/mav-ethereum-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-ethereum-Uniswap-v3-WETH-USDC.paraquet' },
    'arbitrum': {  'nr': '2', 'file_name': './data/mav-arbitrum-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-arbitrum-Uniswap-v3-WETH-USDC.paraquet'},
    'arbitrum2': { 'nr': '3', 'file_name': './data/mav-arbitrum-Uniswap-v3-WETH-USDCe.paraquet', 'output_file': './data/mav2-arbitrum-Uniswap-v3-WETH-USDCe.paraquet'},
    'base': {      'nr': '4', 'file_name': './data/mav-base-Uniswap-v3-WETH-USDC.paraquet'     , 'output_file': './data/mav2-base-Uniswap-v3-WETH-USDC.paraquet'},
    'optimism': {  'nr': '5', 'file_name': './data/mav-optimism-Uniswap-v3-WETH-USDC.paraquet' , 'output_file': './data/mav2-optimism-Uniswap-v3-WETH-USDC.paraquet'},
    'zksync': {    'nr': '6', 'file_name': './data/mav-zksync-Uniswap-v3-WETH-USDC.paraquet'   , 'output_file': './data/mav2-zksync-Uniswap-v3-WETH-USDC.paraquet' } ,
}

#columns = ["Binance","Ethereum","Arbitrum","Arbitrum(e)","Base", "Optimism(e)", "zkSync(e)"]
#price_diff_matrix = np.zeros((7, 7))
#MAV_matrix = np.zeros((7, 7))

In [4]:

for key0, value0 in file_settings.items():
    file_name0 = value0['file_name']
    output_file0 = value0['output_file']
    suffix0 = "_" + value0['nr']
    print("loop: " + value0['nr'])

    #cleaning
    swaps_df0 = pl.scan_parquet(file_name0).collect(streaming=True)
    swaps_df0.write_parquet(output_file0)
    del swaps_df0

    #start calculations
    for key1, value1 in file_settings.items():

        swaps_df0 = pl.scan_parquet(output_file0).collect(streaming=True)

        suffix = "_" + value1['nr']
        if (suffix0 != suffix) :
            print(suffix)
            file_name1 = value1['file_name']

            #swaps_df1 = pl.scan_parquet(file_name1).collect(streaming=True)
            swaps_df_full = pl.scan_parquet(file_name1).collect(streaming=True)
            swaps_df_full = swaps_df0.join(swaps_df_full, on='block_timestamp', how="outer", suffix=suffix)
            #del swaps_df1

            swaps_df_full = swaps_df_full.with_columns(
                pl.when(pl.col("block_timestamp").is_not_null())
                .then(pl.col("block_timestamp") )
                .otherwise(pl.col("block_timestamp"+suffix) )
                .alias("block_timestamp_X")
            )
            swaps_df_full = swaps_df_full.sort("block_timestamp_X")

            swaps_df_full = swaps_df_full.with_columns(pl.col("spot_price"+suffix).fill_null(strategy="forward"))
            swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_USD"+suffix).fill_null(strategy="forward"))
            swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_ETH"+suffix).fill_null(strategy="forward"))
            swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_total_USD"+suffix).fill_null(strategy="forward"))

            swaps_df_full = swaps_df_full.select("block_timestamp", "spot_price"+suffix, "reserve_USD"+suffix, "reserve_ETH"+suffix, "reserve_total_USD"+suffix)
            swaps_df0 = swaps_df0.join(swaps_df_full, on = "block_timestamp", how="left")

            del swaps_df_full
        swaps_df0.write_parquet(output_file0)
        del swaps_df0
        


loop: 1
_2
_3


In [ ]:
swaps_df0.columns fhffh gg

SyntaxError: invalid syntax (3795058954.py, line 1)

In [ ]:

swaps_df_full = swaps_df_full.with_columns(
    pl.when(pl.col("block_timestamp").is_not_null())
    .then(pl.col("block_timestamp") )
    .otherwise(pl.col("block_timestamp_6") )
    .alias("block_timestamp_X")
)

swaps_df_full

In [ ]:
swaps_df_full = swaps_df_full.sort("block_timestamp_X")
swaps_df_full


In [ ]:
my_swaps = swaps_df_full.filter(pl.col('block_timestamp_6').is_not_null())

In [ ]:
my_swaps

In [ ]:
swaps_df_full = swaps_df_full.with_columns(pl.col("spot_price_6").fill_null(strategy="forward"))

swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_USD_6").fill_null(strategy="forward"))
swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_ETH_6").fill_null(strategy="forward"))
swaps_df_full = swaps_df_full.with_columns(pl.col("reserve_total_USD_6").fill_null(strategy="forward"))



In [ ]:
swaps_df_full = swaps_df_full.select("block_timestamp", "spot_price_6", "reserve_USD_6", "reserve_ETH_6", "reserve_total_USD_6")
swaps_df_full

In [ ]:
swaps_df01 = swaps_df0.join(swaps_df_full, on = "block_timestamp", how="left")

swaps_df01